### Imports

In [4]:
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import cvxpy as cp 

Question c)

In [9]:
from scipy.optimize import minimize, NonlinearConstraint, Bounds
from numpy import sqrt

# define constants given bar length L, external force, saftey factor s
L = 12.5    # length of the beam [cm]
A = 100     # [cm^3]
B = 8       # [cm^2]
C = 3000    # [cm^5]

# define objective function
def f(x):
    return x[0]**2 * L + x[1]**2 * sqrt(L**2 + x[2]**2)

# define constraints
g1 = lambda x: A / (x[0]**2 * x[2]) - 1
g2 = lambda x: B * sqrt(L**2 + x[2]**2) / (x[1]**2 * x[2]) - 1
g3 = lambda x: C / (x[0]**4 * x[2]) - 1

g1_con = NonlinearConstraint(g1, -np.inf, 0)
g2_con = NonlinearConstraint(g2, -np.inf, 0)
g3_con = NonlinearConstraint(g3, -np.inf, 0)

# define bounds
bnds = Bounds([0.001, 0.001, 0.001], [np.inf, np.inf, np.inf])

# initial guess
x0 = np.array([1, 1, 1])

# solve optimization problem
res = minimize(f, x0, method='trust-constr', constraints=[g1_con, g2_con, g3_con], bounds=bnds)

# print results
print(res)

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 80
      cg_stop_cond: 4
            constr: [array([-0.57482205]), array([-0.]), array([-0.]), array([ 3.57146166,  3.10885693, 18.43899674])]
       constr_nfev: [192, 192, 192, 0]
       constr_nhev: [0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.10271501541137695
               fun: 374.74481046719563
              grad: array([ 89.28654216, 138.50948262,   8.        ])
               jac: [array([[-0.23809744,  0.        , -0.02305863]]), array([[ 0.        , -0.64332325, -0.01707596]]), array([[-1.11998958,  0.        , -0.05423288]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])]
   lagrangian_grad: array([ 0.        ,  0.        , -0.00000006])
           message: '`xtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 192
              nhev

c:\Users\lange\.conda\envs\4dm20\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


We read: `v: [0., 215.3, 79.72]`

These are the lagrange multipliers. Thus implying g1 is inactive, g2 is active, g3 is active.

Furthermore we have: $x^* = [3.57,  3.11, 18.44]$

In [10]:
A

100

Question f) verify the KKT conditions:

$$\begin{align}
\nabla f(x^*) + \mu^{*T}\nabla g(x^*) &= 0 & \text{Optimality}\\
g(x^*) &\leq 0 & \text{Feasibility}\\
\mu(x^*)^T g(x^*) &= 0 & \text{Complementary slackness}\\
\end{align}$$

We have:
$ x^* = [3.57,  3.11, 18.44]$, $ \mu^* = [0, 215.3, 79.72]$

Constraints at $x^*$:

$$\begin{align}
g_1(x^*) &= \frac{A}{x_1^2 x_3} - 1 = \frac{100}{3.57^2 \cdot 18.44} - 1 = -0.5745 < 0\\
g_2(x^*) &= \frac{B \sqrt{L^2 + x_3^2}}{x_2^2 x_3} - 1 = \frac{8 \sqrt{12.5^2 + 18.44^2}}{3.11^2 \cdot 18.44} - 1 = 0\\
g_3(x^*) &= \frac{C}{x_1^4x_3} -1 = \frac{3000}{3.57^4 \cdot 18.44} - 1 = -0.5745 = 0\\
\end{align}$$

So this is correct, g2 and g3 are active, g1 is inactive. Complementary slackness is also satisfied, since $\mu(x^*)^T g(x^*) = 0$

Lastly, we verify the gradient condition:
...